In [104]:
import sys
!{sys.executable} -m pip install matplotlib pandas xlrd seaborn tqdm scikit-learn tensorflow openpyxl

  Using cached https://files.pythonhosted.org/packages/bf/58/34bfa8fa17f86333361172b3b502e805195180f19a7496ad0f6149138d55/tensorflow-1.13.1-cp36-cp36m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/0f/39/bdd75b08a6fba41f098b6cb091b9e8c7a80e1b4d679a581a0ccd17b10373/tensorboard-1.13.1-py3-none-any.whl
  Running setup.py bdist_wheel for openpyxl: started
  Running setup.py bdist_wheel for openpyxl: finished with status 'done'
  Stored in directory: C:\Users\shchukin\AppData\Local\pip\Cache\wheels\73\73\c8\bbc0e4a79fc525440783741909e4e47024430f082fc7e0936f
  Running setup.py bdist_wheel for et-xmlfile: started
  Running setup.py bdist_wheel for et-xmlfile: finished with status 'done'
  Stored in directory: C:\Users\shchukin\AppData\Local\pip\Cache\wheels\2a\77\35\0da0965a057698121fc7d8c5a7a9955cdbfb3cc4e2423cad39
Successfully built openpyxl et-xmlfile
  Found existing installation: tensorboard 1.12.2
    Uninstalling tensorboard-1.12.2:
      Successfully uninstalled 

Could not install packages due to an EnvironmentError: [WinError 5] Отказано в доступе: 'c:\\users\\shchukin\\appdata\\local\\programs\\python\\python36\\Lib\\site-packages\\tensorflow\\python\\_pywrap_tensorflow_internal.pyd'
Consider using the `--user` option or check the permissions.

You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tqdm
import numpy as np

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, BatchNormalization, LeakyReLU, Activation, Dropout
from tensorflow.python.keras.layers.recurrent import LSTM
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.python.keras.regularizers import l2

Чтение данных из Excel

In [2]:
sales = pd.read_excel('ДанныеДекабря_2017.xlsx', 'Продажи')
sales = sales.set_index('Material')

ap = pd.read_excel('ДанныеДекабря_2017.xlsx', 'Магазины')
ap = ap.set_index('Material')

stock = pd.read_excel('ДанныеДекабря_2017.xlsx', 'Остатки')
stock = stock.set_index('Material')

price = pd.read_excel('ДанныеДекабря_2017.xlsx', 'Цены')
price = price.set_index('Material')

# hierarchy = pd.read_excel('hierarchy.xlsx')

# sell = pd.read_excel('Акции.xlsx')

Создание DataFrame для анализа зависимостей между переменными по артикулам

In [3]:
data_df = pd.melt(sales.reset_index(), id_vars=['Material'], value_vars=list(sales.columns), var_name='date', value_name='sales').sort_values(by=['Material','date'], ascending=False)
data_df = pd.merge(data_df, pd.melt(stock.reset_index(), id_vars=['Material'], value_vars=list(stock.columns), var_name='date', value_name='stock_1').sort_values(by=['Material','date'], ascending=False).shift(-1), on=['Material', 'date'])
data_df = pd.merge(data_df, pd.melt(price.reset_index(), id_vars=['Material'], value_vars=list(price.columns), var_name='date', value_name='price_1').sort_values(by=['Material','date'], ascending=False).shift(-1), on=['Material', 'date'])
data_df = pd.merge(data_df, pd.melt(ap.reset_index(), id_vars=['Material'], value_vars=list(ap.columns), var_name='date', value_name='ap_1').sort_values(by=['Material','date'], ascending=False).shift(-1), on=['Material', 'date'])
data_df.loc[:,'sales_1'] = data_df['sales'].shift(-1)
data_df.loc[:,'sales_2'] = data_df['sales_1'].shift(-1)
data_df.loc[:,'sales_3'] = data_df['sales_1'].shift(-2)
data_df.loc[:,'sales_11'] = data_df['sales_1'].shift(-10)
data_df.loc[:,'sales_12'] = data_df['sales_1'].shift(-11)
data_df.loc[:,'sales_13'] = data_df['sales_1'].shift(-12)
data_df.loc[:,'sales_trend_1'] = (data_df['sales_1'] - data_df['sales_1'].shift(-1)) / data_df['sales_1']
data_df.loc[:,'sales_trend_2'] = (data_df['sales_1'].shift(-1) - data_df['sales_1'].shift(-2)) / data_df['sales_1']
data_df.loc[:,'sales_trend_ly'] = ((data_df['sales_1'] + data_df['sales_1'].shift(-1) + data_df['sales_1'].shift(-2)) - (data_df['sales_1'] + data_df['sales_1'].shift(-12) + data_df['sales_1'].shift(-14))) / (data_df['sales_1'] + data_df['sales_1'].shift(-1) + data_df['sales_1'].shift(-2))
data_df.loc[:,'stock_2'] = data_df['stock_1'].shift(-1)
data_df.loc[:,'stock_3'] = data_df['stock_1'].shift(-2)
data_df.loc[:,'stock_trend_1'] = (data_df['stock_1'] - data_df['stock_1'].shift(-1)) / data_df['stock_1']
data_df.loc[:,'stock_trend_2'] = (data_df['stock_1'].shift(-1) - data_df['stock_1'].shift(-2)) / data_df['stock_1']
data_df.loc[:,'price_2'] = data_df['price_1'].shift(-1)
data_df.loc[:,'price_3'] = data_df['price_1'].shift(-2)
data_df.loc[:,'price_trend_1'] = (data_df['price_1'] - data_df['price_1'].shift(-1)) / data_df['price_1']
data_df.loc[:,'price_trend_2'] = (data_df['price_1'].shift(-1) - data_df['price_1'].shift(-2)) / data_df['price_1']
data_df.loc[:,'ap_2'] = data_df['ap_1'].shift(-1)

data_df = data_df.set_index('date')
    
data_df.loc[:,'month'] = data_df.index.month

data_df = data_df.replace([np.inf, -np.inf], np.nan)

data_df = data_df.dropna()

In [5]:
del sales
del ap
del stock
del price

NameError: name 'sales' is not defined

Сохраняем собранные данные в файл для ускорения в будущем

In [6]:
data_df.to_excel('full_train_data.xlsx')

Выгружаем данные для обучения из файла

In [16]:
data_df = pd.read_excel('full_train_data.xlsx')
data_df = data_df.set_index('date')

Создаем DF корелляции

In [17]:
corr_df = data_df.groupby('Material').corr().abs().sales.fillna(0).unstack(level=0).T

Выделение групп артикулов для ращного прогнозирования на основе коррелляции

In [18]:
#Cluster the data
kmeans = KMeans(n_clusters=5, random_state=0).fit(corr_df)
labels = kmeans.labels_

#Glue back to originaal data
corr_df['clusters'] = labels

# Создание нормализованной DF
corr_overal_df = corr_df.groupby('clusters').mean()

corr_overal_norm_df = (corr_overal_df-corr_overal_df.min())/(corr_overal_df.max()-corr_overal_df.min())

In [10]:
corr_df.to_excel('material_clusters.xlsx')

In [19]:
corr_overal_df.style.background_gradient(cmap='viridis')

,ap_1,ap_2,month,price_1,price_2,price_3,price_trend_1,price_trend_2,sales,sales_1,sales_11,sales_12,sales_13,sales_2,sales_3,sales_trend_1,sales_trend_2,sales_trend_ly,stock_1,stock_2,stock_3,stock_trend_1,stock_trend_2
clusters,,,,,,,,,,,,,,,,,,,,,,,
0,0.734709,0.660481,0.604239,0.486847,0.418835,0.560024,0.427686,0.56693,1,0.393427,0.10222,0.0576844,0.0349094,0.442335,0.509084,0.512623,0.345622,0.613996,0.708776,0.573164,0.497616,0.514919,0.604551
1,0.768471,0.744041,0.310554,0.394502,0.311505,0.246264,0.228897,0.212046,1,0.597541,0.419203,0.401275,0.333808,0.45702,0.403134,0.184001,0.143866,0.25434,0.741432,0.680289,0.532365,0.263638,0.284268
2,0.293494,0.235834,0.259963,0.359155,0.225196,0.185235,0.290194,0.201214,0.939286,0.20961,0.232671,0.207437,0.180007,0.162709,0.173792,0.167591,0.148927,0.176816,0.223064,0.196617,0.192783,0.210093,0.197303
3,0.910933,0.861534,0.968053,0.903158,0.860602,0.933156,0.887336,0.955124,1,0.789669,0.0305165,0.0135087,0.0215085,0.92309,0.273443,0.931583,0.833383,0.96031,0.937891,0.909647,0.957667,0.929991,0.956538
4,0.630145,0.584894,0.278819,0.299204,0.248329,0.202988,0.201681,0.184554,1,0.353252,0.177393,0.208201,0.156749,0.216763,0.20822,0.198335,0.156046,0.233497,0.608834,0.49548,0.311932,0.256215,0.302923


Вычисление колонок у кластера для использования для прогнозирования

In [20]:
# Будем отсекать по средней корелляции, за исключением артикулов, у которых нет данных
threshold = corr_overal_df[corr_overal_df.sales > 0.1].mean()
# Инициализация копии DF
corr_thresh_df = corr_overal_df.copy()
# Прогон сравнения по всем колонкам
for col in corr_overal_df.columns:
    corr_thresh_df[col] = (corr_overal_df[col] >= threshold[col]).astype(int)
    
corr_thresh_df.style.background_gradient(cmap='viridis')

,ap_1,ap_2,month,price_1,price_2,price_3,price_trend_1,price_trend_2,sales,sales_1,sales_11,sales_12,sales_13,sales_2,sales_3,sales_trend_1,sales_trend_2,sales_trend_ly,stock_1,stock_2,stock_3,stock_trend_1,stock_trend_2
clusters,,,,,,,,,,,,,,,,,,,,,,,
0,1,1,1,0,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1,1,0,1,1
1,1,1,0,0,0,0,0,0,1,1,1,1,1,1,1,0,0,0,1,1,1,0,0
2,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,1,1,1,1,1,1,0,0,0,1,0,1,1,1,1,1,1,1,1
4,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0


Создаем DF для всех материалов для обучения нейронки

In [21]:
train_data = data_df.merge(corr_df.clusters.to_frame(), how='left', left_on='Material', right_index=True)

Создаем тренировочные данные

In [22]:
X = train_data[[col for col in train_data.columns if col not in ['sales', 'Material']]]

Y = train_data.sales.to_frame()

scaler = MinMaxScaler()

print(scaler.fit(X))
print(scaler.fit(Y))

xscale=scaler.transform(X)
yscale=scaler.transform(Y)

X_train, X_test, y_train, y_test = train_test_split(xscale, yscale)

# # for LSTM
# X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
# X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
# y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], 1))
# y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], 1))

# shape data for LSTM model
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
# y_train = y_train.reshape((y_train.shape[0], 1, y_train.shape[1]))
# y_test = y_test.reshape((y_test.shape[0], 1, y_test.shape[1]))

c:\python36\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


MinMaxScaler(copy=True, feature_range=(0, 1))


c:\python36\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


MinMaxScaler(copy=True, feature_range=(0, 1))


In [23]:
# # 46.8%
# model = Sequential()
# model.add(Dense(128, input_dim=23, kernel_initializer='normal'))
# model.add(Dense(64))
# model.add(Dense(16))
# model.add(Dense(1))


# # Trading
# # 25% переобучение
# model = Sequential()
# model.add(Dense(64, input_dim=23,activity_regularizer=l2(0.01)))
# model.add(BatchNormalization())
# model.add(LeakyReLU())
# model.add(Dense(16,activity_regularizer=l2(0.01)))
# model.add(BatchNormalization())
# model.add(LeakyReLU())
# model.add(Dense(1))
# model.add(Activation('linear'))


# RNN
model = Sequential()
model.add(LSTM(64,  input_shape=(1, X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=64, return_sequences=True))  
model.add(Dropout(0.2))
model.add(LSTM(units=64)) 
model.add(Dropout(0.2)) 
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# Compile
model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 64)             22528     
_________________________________________________________________
dropout (Dropout)            (None, 1, 64)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 64)             33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 64)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_______________________________________

In [ ]:
history = model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

Instructions for updating:
Use tf.cast instead.


In [ ]:
plt.figure()
plt.plot(history.history['mean_squared_error'])
# plt.plot(history.history['mean_absolute_error'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['mse', 'mae', 'mape', 'cp'], loc='best')
plt.show()

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
prediction = model.predict(X_test)

prediction = prediction.reshape((prediction.shape[0], 1))
test = y_test.reshape((y_test.shape[0], 1))

accuracy = 1 - sum(abs(scaler.inverse_transform(prediction) - scaler.inverse_transform(test))) / sum(scaler.inverse_transform(test))

print('Total accuracy = %.2f%%' % (accuracy * 100))

In [33]:
# Сохраним модель полностью в единый HDF5 файл

In [34]:
model.save('best_model.h5')